In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
import warnings
warnings.simplefilter("ignore")
from sklearn.cluster import KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.preprocessing import MinMaxScaler

In [2]:
wm = pd.read_csv(r'WineMatters.csv')
wm.head(10)

,WineID,Aging_Time,Litters_Barrel,Type,Magnesium,Residual_Sugar,Sulphites,Barrel,Grapes,Acidity,...,Red_Fruit,Citric,Density,Color_Intensity,Cloudiness,Alcohol,Astringency,Satisfaction_France,Satisfaction_Spain,Satisfaction_Portugal
0,1,65,265,White,0,0.0,Not Present,Aluminium,Single,0,...,2,4,2,3,3,0,0,6,6,4
1,2,47,2464,Red,310,305.0,Present,Aluminium,Single,4,...,0,2,2,3,4,0,0,8,6,4
2,3,15,2138,White,0,0.0,Not Present,Aluminium,Single,3,...,2,0,2,2,3,0,0,8,8,4
3,4,60,623,White,0,0.0,Not Present,Aluminium,Single,0,...,3,4,3,1,1,0,0,2,2,6
4,5,70,354,White,0,0.0,Not Present,Aluminium,Single,0,...,4,3,4,2,2,0,0,4,4,10
5,6,30,1894,Red,0,0.0,Not Present,Aluminium,Single,4,...,2,0,2,2,5,0,0,10,8,4
6,7,66,227,White,17,15.0,Not Present,Aluminium,Single,0,...,2,5,5,5,5,0,0,10,10,6
7,8,10,1812,Red,0,0.0,Not Present,Aluminium,Single,3,...,2,0,2,2,3,0,0,8,8,4
8,9,56,73,White,0,0.0,Present,Aluminium,Single,0,...,5,3,5,4,4,0,0,2,8,8
9,10,22,1556,Red,30,26.0,Not Present,Aluminium,Single,4,...,2,0,2,2,2,0,0,10,8,4


In [5]:
# Set WineID as an index
wm.set_index('WineID', inplace = True)

KeyError: "None of ['WineID'] are in the columns"

In [6]:
wm.describe()

,Aging_Time,Litters_Barrel,Magnesium,Residual_Sugar,Acidity,Floral,Wood,Sweetness,Red_Fruit,Citric,Density,Color_Intensity,Cloudiness,Alcohol,Astringency,Satisfaction_France,Satisfaction_Spain,Satisfaction_Portugal
count,129881.000000,129881.000000,129881.000000,129490.000000,129881.000000,129881.000000,129881.000000,129881.000000,129881.000000,129881.000000,129881.000000,129881.000000,129881.000000,129881.000000,129881.000000,129881.000000,129881.000000,129881.000000
mean,39.427907,1981.419854,14.713599,15.090779,3.485914,3.340820,2.851972,2.990422,3.249113,3.383451,3.519683,3.472086,3.466743,2.838575,2.990653,7.391351,7.411523,6.705138
std,15.119312,1027.119025,38.071002,38.465273,1.292228,1.260586,1.443746,1.305965,1.318827,1.346087,1.306524,1.305573,1.273503,1.393000,1.527221,2.312959,2.303540,2.597452
min,7.000000,50.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000
25%,27.000000,1359.000000,0.000000,0.000000,2.000000,3.000000,2.000000,2.000000,2.000000,2.000000,3.000000,2.000000,3.000000,2.000000,2.000000,6.000000,6.000000,4.000000
50%,40.000000,1925.000000,0.000000,0.000000,4.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,3.000000,3.000000,8.000000,8.000000,8.000000
75%,51.000000,2544.000000,12.000000,13.000000,5.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,5.000000,4.000000,4.000000,4.000000,10.000000,10.000000,8.000000
max,85.000000,6951.000000,1592.000000,1584.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,6.000000,5.000000,5.000000,10.000000,10.000000,10.000000


In [8]:
wm.shape

(129881, 22)

In [9]:
#Check for enchoerences
wm['Cloudiness',].max()

6

In [10]:
# Redefine values that are higher than 5 to 5
wm.loc[wm['Cloudiness']>5,'Cloudiness']=5
wm['Cloudiness'].max()

5

In [12]:
#Cleaning outliers Magnesium
wm_after_Magnesium=wm[wm['Magnesium'] < 440]
wm_after_Magnesium.shape

(129811, 22)

In [13]:
#Cleaning outliers Residual Sugar
wm_after_ResidualSugar=  wm_after_Magnesium[wm_after_Magnesium['Residual_Sugar'] < 450]
wm_after_ResidualSugar.shape

(129417, 22)

In [14]:
#Cleaning outliers Litters per Barrel
wm_after_outliers=  wm_after_ResidualSugar[wm_after_ResidualSugar['Litters_Barrel'] < 6420]
wm_after_outliers.shape

(129375, 22)

In [18]:
wm_cleaned=wm_after_outliers

In [20]:
wm_cleaned[wm_cleaned.duplicated(keep=False)]

,Aging_Time,Litters_Barrel,Type,Magnesium,Residual_Sugar,Sulphites,Barrel,Grapes,Acidity,Floral,...,Red_Fruit,Citric,Density,Color_Intensity,Cloudiness,Alcohol,Astringency,Satisfaction_France,Satisfaction_Spain,Satisfaction_Portugal
WineID,,,,,,,,,,,,,,,,,,,,,
354,33,3384,White,0,0.0,Very Few,Aluminium,Single,5,5,...,1,0,1,1,3,0,4,8,8,2
354,33,3384,White,0,0.0,Very Few,Aluminium,Single,5,5,...,1,0,1,1,3,0,4,8,8,2


In [21]:
wm_cleaned.drop_duplicates(inplace = True)
wm_cleaned[wm_cleaned.duplicated(keep=False)]

,Aging_Time,Litters_Barrel,Type,Magnesium,Residual_Sugar,Sulphites,Barrel,Grapes,Acidity,Floral,...,Red_Fruit,Citric,Density,Color_Intensity,Cloudiness,Alcohol,Astringency,Satisfaction_France,Satisfaction_Spain,Satisfaction_Portugal
WineID,,,,,,,,,,,,,,,,,,,,,


In [23]:
# Check sum of missing value per variable
wm_cleaned.isna().sum()

# We have missing values for, Barrel and Grapes
#the missing values for the outliers were deleted when the outliers were deleted 

Aging_Time                0
Litters_Barrel            0
Type                      0
Magnesium                 0
Residual_Sugar            0
Sulphites                 0
Barrel                   56
Grapes                    8
Acidity                   0
Floral                    0
Wood                      0
Sweetness                 0
Red_Fruit                 0
Citric                    0
Density                   0
Color_Intensity           0
Cloudiness                0
Alcohol                   0
Astringency               0
Satisfaction_France       0
Satisfaction_Spain        0
Satisfaction_Portugal     0
dtype: int64

In [26]:
#does not make sence to fill the missing values for barrel and grapes because they are categorical values so we delete them
wm_cleaned = wm_cleaned.dropna(axis = 0, how ='any')
wm_cleaned.isna().sum()

Aging_Time               0
Litters_Barrel           0
Type                     0
Magnesium                0
Residual_Sugar           0
Sulphites                0
Barrel                   0
Grapes                   0
Acidity                  0
Floral                   0
Wood                     0
Sweetness                0
Red_Fruit                0
Citric                   0
Density                  0
Color_Intensity          0
Cloudiness               0
Alcohol                  0
Astringency              0
Satisfaction_France      0
Satisfaction_Spain       0
Satisfaction_Portugal    0
dtype: int64

In [27]:
wm_cleaned.shape

(129426, 22)

In [28]:
#Those are the variables that we are working with
mw_flavfeel = wm_cleaned[['Residual_Sugar','Acidity','Floral','Wood','Sweetness','Red_Fruit','Citric','Density','Cloudiness','Alcohol','Astringency']].copy()
mw_flavfeel

,Residual_Sugar,Acidity,Floral,Wood,Sweetness,Red_Fruit,Citric,Density,Cloudiness,Alcohol,Astringency
WineID,,,,,,,,,,,
1,0.0,0,5,0,2,2,4,2,3,0,0
2,305.0,4,2,0,3,0,2,2,4,0,0
3,0.0,3,4,0,3,2,0,2,3,0,0
4,0.0,0,4,0,3,3,4,3,1,0,0
5,0.0,0,4,0,3,4,3,4,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...
129876,0.0,3,4,5,3,2,5,2,3,5,5
129877,172.0,3,1,2,4,2,1,1,2,2,3
129878,163.0,3,2,3,3,3,2,2,4,3,0
